In [19]:
import os
import sys
cur_dir = os.path.dirname(os.path.abspath('__file__'))
sys.path.insert(0, os.path.abspath("%s/../clipper_admin" % cur_dir))
import logging, xgboost as xgb, numpy as np
from clipper_admin import ClipperConnection, DockerContainerManager
cl = ClipperConnection(DockerContainerManager())
cl.start_clipper()
# We will register it to deploy an xgboost model.
cl.register_application('xgboost-test', 'integers', 1000000000, 'enemble')

18-07-06:19:54:34 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-07-06:19:54:38 INFO     [clipper_admin.py:126] Clipper is running
18-07-06:19:54:38 ERROR    [clipper_admin.py:179] HELLO
18-07-06:19:54:38 INFO     [clipper_admin.py:190] {"name": "xgboost-test", "input_type": "integers", "selection_policy": "enemble", "latency_slo_micros": 1000000000}
18-07-06:19:54:38 INFO     [clipper_admin.py:201] Application xgboost-test was successfully registered


In [20]:
def get_test_point():
    return [np.random.randint(255) for _ in range(7)]
# Create a training matrix.
dtrain = xgb.DMatrix(get_test_point(), label=[0])
# We then create parameters, watchlist, and specify the number of rounds
# This is code that we use to build our XGBoost Model, and your code may differ.
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
watchlist = [(dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)
def predict(xs):
    b = bst.predict(xgb.DMatrix(xs))
    print(b)
    return b
from clipper_admin.deployers import python as python_deployer
# We specify which packages to install in the pkgs_to_install arg.
# For example, if we wanted to install xgboost and psycopg2, we would use
# pkgs_to_install = ['xgboost', 'psycopg2']
python_deployer.deploy_python_closure(cl, name='xgboost-model', version=1,
     input_type="integers", func=predict, pkgs_to_install=['xgboost'])
cl.link_model_to_app('xgboost-test', 'xgboost-model')

[0]	train-error:0
[1]	train-error:0


18-07-06:19:54:38 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpjr5_iq_l
18-07-06:19:54:38 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-07-06:19:54:38 INFO     [python.py:192] Python closure saved
18-07-06:19:54:38 INFO     [python.py:206] Using Python 3.6 base image
18-07-06:19:54:38 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpjr5_iq_l
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python36-closure-container:develop'}
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': ' ---> 8a50905462bc\n'}
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmpjr5_iq_l /model/'}
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-06:19:54:50 INFO     [clipper_admin.py:456] {'stream': ' ---> 80ba97d05a3b\n'}
18-07-06:19:54:5

In [21]:
bost = xgb.train(param, xgb.DMatrix(get_test_point(), label=[0]), num_round, watchlist)
def preict(xs):
    a = bost.predict(xgb.DMatrix(xs))
    print(a)
    return [1 + x for x in a]
python_deployer.deploy_python_closure(cl, name='xgboost-modl', version=1,
     input_type="integers", func=preict, pkgs_to_install=['xgboost'])
cl.link_model_to_app('xgboost-test', 'xgboost-modl')

[0]	train-error:0
[1]	train-error:0


18-07-06:19:54:57 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpaz_mwmak
18-07-06:19:54:57 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-07-06:19:54:57 INFO     [python.py:192] Python closure saved
18-07-06:19:54:57 INFO     [python.py:206] Using Python 3.6 base image
18-07-06:19:54:57 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpaz_mwmak
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python36-closure-container:develop'}
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': ' ---> 8a50905462bc\n'}
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmpaz_mwmak /model/'}
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-06:19:55:08 INFO     [clipper_admin.py:456] {'stream': ' ---> c99fe4cc0ea0\n'}
18-07-06:19:55:0

In [22]:
import requests, json
# Get Address
addr = cl.get_query_addr()
# Post Query
response = requests.post(
     "http://%s/%s/predict" % (addr, 'xgboost-test'),
     headers={"Content-type": "application/json"},
     data=json.dumps({
         'input': get_test_point()
     }))
result = response.json()
result

{'combiner_output': {'combine_error': "Combine Policy not found. Default used.'",
  'final_prediction': [0.5, 1.5],
  'msg': 'return'},
 'models_used': [{'id': '1', 'name': 'xgboost-modl'},
  {'id': '1', 'name': 'xgboost-model'}],
 'query_id': 0}

In [23]:
print(json.dumps(result, indent=4))

{
    "query_id": 0,
    "combiner_output": {
        "msg": "return",
        "combine_error": "Combine Policy not found. Default used.'",
        "final_prediction": [
            0.5,
            1.5
        ]
    },
    "models_used": [
        {
            "name": "xgboost-modl",
            "id": "1"
        },
        {
            "name": "xgboost-model",
            "id": "1"
        }
    ]
}


In [24]:
cl.stop_all()

18-07-06:19:57:00 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
